In [4]:
# Mount drive for ease of using git
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# clone code needed
!git clone https://github.com/mtfishman/ITensorQTT.jl.git

Cloning into 'ITensorQTT.jl'...
remote: Enumerating objects: 482, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 482 (delta 11), reused 10 (delta 4), pack-reused 456
Receiving objects: 100% (482/482), 722.13 KiB | 11.46 MiB/s, done.
Resolving deltas: 100% (242/242), done.


In [6]:
# Install Julia from install page, and add IJulia, which is the julia jupyter kernel
%%capture
%%shell
if ! command -v julia 3>&1 > /dev/null
then
    wget -q 'https://julialang-s3.julialang.org/bin/linux/x64/1.9/julia-1.9.1-linux-x86_64.tar.gz' \
        -O /tmp/julia.tar.gz
    tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
    rm /tmp/julia.tar.gz
fi
julia -e 'using Pkg; pkg"add IJulia; precompile;"'
echo 'Done'

Now switch runtime to Julia

In [3]:
using Pkg
Pkg.add("ITensors")
Pkg.add("ITensorTDVP")
Pkg.add("FFTW")
Pkg.add("Polynomials")
Pkg.add("KrylovKit")
using ITensors
using FFTW

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


In [4]:
include("./ITensorQTT.jl/src/ITensorQTT.jl")
using .ITensorQTT

In [5]:
Pkg.add("TimerOutputs")
using TimerOutputs

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


In [6]:
Pkg.add("PyCall")
using PyCall

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


In [7]:
using DelimitedFiles

In [8]:
ITensors.set_warn_order(30)

14

In [9]:
# TimingDriver: n_qubits = number of qubits
# func = function name
#    -options = [gaussian, cos, cusps, fft]
# _cutoff = cutoff during mps and mpo construction
# _to = TimerOutput that saves the runtimes

function TimingDriver(n_qubits::Int64, _func::String, _cutoff::Float64, _to)
  s = siteinds("Qubit", n_qubits)
  if _func == "gaussian"
    x_not = 0.5
    sigma = 0.1
    f1(x) = exp(-(x - x_not)^2 / sigma^2)
    psi = @timeit _to "gaus: function_to_mps: $n_qubits" function_to_mps(f1, s, 0.0, 1.0; cutoff=_cutoff)
    f_psi = @timeit _to "gaus: apply_dft_mpo: $n_qubits" apply_dft_mpo(psi; cutoff=_cutoff)
  elseif _func == "cos"
    f2(x) = cos(2*pi*x)
    psi = @timeit _to "cos: function_to_mps: $n_qubits" function_to_mps(f2, s, 0.0, 1.0; cutoff=_cutoff)
    f_psi = @timeit _to "cos: apply_dft_mpo: $n_qubits" apply_dft_mpo(psi; cutoff=_cutoff)
  elseif _func == "cusps"
    f3(x) = cos(2*π*x) + 2*exp(-3*abs(x-0.2))
    psi = @timeit _to "cusps: function_to_mps: $n_qubits" function_to_mps(f3, s, 0.0, 1.0; cutoff=_cutoff)
    f_psi = @timeit _to "cusps: apply_dft_mpo: $n_qubits" apply_dft_mpo(psi; cutoff=_cutoff)
  elseif _func == "fft"
    @timeit _to "fft: $n_qubits" fft([x for x=0:(2^n_qubits)])
  end
  return
end

TimingDriver (generic function with 1 method)

In [12]:
# Initialize Timer
to = TimerOutput()
start = 4
stop = 26

# Run TimingDriver for functions from qubits=start to qubits=stop
for i in start:stop
  TimingDriver(i, "gaussian", 1e-15, to)
  TimingDriver(i, "cos", 1e-15, to)
  TimingDriver(i, "cusps", 1e-15, to)
  TimingDriver(i, "fft", 1e-15, to)
end
show(to)

 ────────────────────────────────────────────────────────────────────────────────
                                        Time                    Allocations      
                               ───────────────────────   ────────────────────────
       Tot / % measured:             266s / 100.0%            131GiB / 100.0%    

 Section               ncalls     time    %tot     avg     alloc    %tot      avg
 ────────────────────────────────────────────────────────────────────────────────
 gaus: function_to_...      1    43.4s   16.4%   43.4s   22.7GiB   17.4%  22.7GiB
 cusps: function_to...      1    40.2s   15.1%   40.2s   20.8GiB   15.9%  20.8GiB
 gaus: function_to_...      1    21.7s    8.2%   21.7s   11.4GiB    8.7%  11.4GiB
 cos: function_to_m...      1    19.7s    7.4%   19.7s   12.0GiB    9.2%  12.0GiB
 cusps: function_to...      1    19.3s    7.3%   19.3s   10.4GiB    8.0%  10.4GiB
 fft: 26                    1    18.3s    6.9%   18.3s   2.50GiB    1.9%  2.50GiB
 gaus: function

In [13]:
# Here, we take the runtimes for each function and operation and put them into an array

#=================GAUS================#
gaus_ftm = [TimerOutputs.time(to["gaus: function_to_mps: $x"]) for x=start:stop]
gaus_adm = [TimerOutputs.time(to["gaus: apply_dft_mpo: $x"]) for x=start:stop]

#=================COS=================#
cos_ftm = [TimerOutputs.time(to["cos: function_to_mps: $x"]) for x=start:stop]
cos_adm = [TimerOutputs.time(to["cos: apply_dft_mpo: $x"]) for x=start:stop]

#================CUSPS================#
cusps_ftm = [TimerOutputs.time(to["cusps: function_to_mps: $x"]) for x=start:stop]
cusps_adm = [TimerOutputs.time(to["cusps: apply_dft_mpo: $x"]) for x=start:stop]

#=================FFT=================#
fft_time = [TimerOutputs.time(to["fft: $x"]) for x=start:stop]

23-element Vector{Int64}:
       52183
       63330
       66705
      108357
      189103
      183864
      245313
      423107
      789023
     1250266
     2721757
     3483168
    17997399
    35555560
    51004634
    94574377
   150254372
   334769207
   606150333
  2803152796
  4222148405
  9673084430
 18333345073

In [14]:
# We start by deleting the existing csv containing the times for each function and operation
# We then write to a new csv for each function and operation

#============GAUS================#
run(`rm -f drive/MyDrive/Colab Notebooks/github/julia_qft/gaus_function_to_mps.csv`)
writedlm( "drive/MyDrive/Colab Notebooks/github/julia_qft/gaus_function_to_mps.csv",  gaus_ftm, ',')

run(`rm -f drive/MyDrive/Colab Notebooks/github/julia_qft/gaus_apply_dft_mpo.csv`)
writedlm( "drive/MyDrive/Colab Notebooks/github/julia_qft/gaus_apply_dft_mpo.csv",  gaus_adm, ',')

#============COS=================#
run(`rm -f drive/MyDrive/Colab Notebooks/github/julia_qft/cos_function_to_mps.csv`)
writedlm( "drive/MyDrive/Colab Notebooks/github/julia_qft/cos_function_to_mps.csv",  cos_ftm, ',')

run(`rm -f drive/MyDrive/Colab Notebooks/github/julia_qft/cos_apply_dft_mpo.csv`)
writedlm( "drive/MyDrive/Colab Notebooks/github/julia_qft/cos_apply_dft_mpo.csv",  cos_adm, ',')

#================CUSPS================#
run(`rm -f drive/MyDrive/Colab Notebooks/github/julia_qft/cusps_function_to_mps.csv`)
writedlm( "drive/MyDrive/Colab Notebooks/github/julia_qft/cusps_function_to_mps.csv",  cusps_ftm, ',')

run(`rm -f drive/MyDrive/Colab Notebooks/github/julia_qft/cusps_apply_dft_mpo.csv`)
writedlm( "drive/MyDrive/Colab Notebooks/github/julia_qft/cusps_apply_dft_mpo.csv",  cusps_adm, ',')

#================FFT=================#
run(`rm -f drive/MyDrive/Colab Notebooks/github/julia_qft/fft.csv`)
writedlm( "drive/MyDrive/Colab Notebooks/github/julia_qft/fft.csv",  fft_time, ',')


In [20]:
# Python function that reads from the csv's and saves a graph from matplotlib

py"""
def graph():
  from datetime import datetime
  import matplotlib.pyplot as plt
  import pandas
  plt.clf()

  #=======GAUSSIAN=========
  gaus_adm = pandas.read_csv('drive/MyDrive/Colab Notebooks/github/julia_qft/gaus_apply_dft_mpo.csv', header=None)
  gaus_ftm = pandas.read_csv('drive/MyDrive/Colab Notebooks/github/julia_qft/gaus_function_to_mps.csv', header=None)

  plt.plot(range(4,gaus_adm.shape[0]+4), gaus_adm[0], linestyle='-', marker='^', color='b', label="Gaus: DFT MPO")
  plt.plot(range(4,gaus_adm.shape[0]+4), gaus_adm[0]+gaus_ftm[0], linestyle='--', marker='^', color='b', label="Gaus: DFT MPO + MPS")

  #===========COSINE=================
  cos_adm = pandas.read_csv('drive/MyDrive/Colab Notebooks/github/julia_qft/cos_apply_dft_mpo.csv', header=None)
  cos_ftm = pandas.read_csv('drive/MyDrive/Colab Notebooks/github/julia_qft/cos_function_to_mps.csv', header=None)

  plt.plot(range(4,cos_adm.shape[0]+4), cos_adm[0], linestyle='-', marker='o', color='r', label="Cos: DFT MPO")
  plt.plot(range(4,cos_adm.shape[0]+4), cos_adm[0]+cos_ftm[0], linestyle='--', marker='o', color='r', label="Cos: DFT MPO + MPS")

  #================CUSPS===================
  cusps_adm = pandas.read_csv('drive/MyDrive/Colab Notebooks/github/julia_qft/cusps_apply_dft_mpo.csv', header=None)
  cusps_ftm = pandas.read_csv('drive/MyDrive/Colab Notebooks/github/julia_qft/cusps_function_to_mps.csv', header=None)

  plt.plot(range(4,cusps_adm.shape[0]+4), cusps_adm[0], linestyle='-', marker='s', color='g', label="Cusps: DFT MPO")
  plt.plot(range(4,cusps_adm.shape[0]+4), cusps_adm[0]+cusps_ftm[0], linestyle='--', marker='s', color='g', label="Cusps: DFT MPO + MPS")

  #====================FFT=======================
  fft = pandas.read_csv('drive/MyDrive/Colab Notebooks/github/julia_qft/fft.csv', header=None)

  plt.plot(range(4,fft.shape[0]+4), fft[0], linestyle='-', marker='*', color='k', label="FFT")

  plt.title("")
  plt.yscale(value='log')
  plt.xlabel("Number of qubits")
  plt.ylabel("Time takes (ns)")
  plt.legend()
  #plt.figure(figsize=(12,8))
  plt.savefig(f'drive/MyDrive/Colab Notebooks/github/julia_qft/graph_{datetime.now().strftime("%Y:%m:%d,%H:%M:%S")}.png')
"""

In [21]:
run(`rm -f drive/MyDrive/Colab Notebooks/github/julia_qft/graph.png`)
py"graph"()

In [ ]:
#run(`rm -f graph.png`)
#run(`python temp.py`)

Process(`python temp.py`, ProcessExited(0))